In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from pathlib import Path

2023-10-24 13:42:33.454267: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['gender']
    return ecg_data, label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [5]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-24 13:42:38.487721: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38006 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [6]:
## Resnet from https://github.com/antonior92/automatic-ecg-diagnosis/tree/master
## https://www.nature.com/articles/s41467-020-15432-4

def residual_unit(x, y, n_samples_out, n_filters_out, prefix, kernel_size = 16):
    n_samples_in = y.shape[1]
    downsample = n_samples_in // n_samples_out
    n_filters_in = y.shape[2]
    
    if downsample == 1:
        y = y
    else:
        y = layers.MaxPooling1D(downsample, strides=downsample, padding='same', name = f"{prefix}_mp_opt")(y)
        
    if n_filters_in != n_filters_out:
        y = layers.Conv1D(n_filters_out, 1, padding='same', use_bias=False, name = f"{prefix}_conv_opt")(y)
        
    x = layers.Conv1D(n_filters_out, kernel_size, padding='same', use_bias=False, name = f"{prefix}_conv1")(x)
    x = layers.BatchNormalization(name = f"{prefix}_bn1")(x)
    x = layers.Activation("relu", name = f"{prefix}_act1")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout1")(x)
    x = layers.Conv1D(n_filters_out, kernel_size, strides=downsample, padding='same', use_bias=False, name = f"{prefix}_conv2")(x)

    x = layers.Add(name = f"{prefix}_add")([x,y])
    y = x
    x = layers.BatchNormalization(name = f"{prefix}_bn2")(x)
    x = layers.Activation("relu", name = f"{prefix}_act2")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout2")(x)
    return (x,y)
    
    
    

input_layer = tf.keras.layers.Input(shape=(5000,12), name="input")
x = input_layer
x = layers.Conv1D(64, 16, padding='same', use_bias=False, name = "conv_1")(x)
x = layers.BatchNormalization(name="bn")(x)
x = layers.Activation("relu", name="relu")(x)

x, y = residual_unit(x,x,1024,128, "res1")
x, y = residual_unit(x,y,256,196, "res2")
x, y = residual_unit(x,y,64,256, "res3")
x, _ = residual_unit(x,y,16,320, "res4")

x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 5000, 12)]           0         []                            
                                                                                                  
 conv_1 (Conv1D)             (None, 5000, 64)             12288     ['input[0][0]']               
                                                                                                  
 bn (BatchNormalization)     (None, 5000, 64)             256       ['conv_1[0][0]']              
                                                                                                  
 relu (Activation)           (None, 5000, 64)             0         ['bn[0][0]']                  
                                                                                              

In [8]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "resnet-gender"))
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-24 13:42:42.550286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-10-24 13:42:44.119141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-24 13:42:44.146271: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f223cf93580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-24 13:42:44.146304: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2023-10-24 13:42:44.219264: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-24 13:42:44.684790: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

   7851/Unknown - 372s 46ms/step - loss: 0.4969 - accuracy: 0.7596 - auc: 0.8399

2023-10-24 13:48:51.952351: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 4976766427408302011
2023-10-24 13:48:51.954558: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3301936778090192773
2023-10-24 13:49:25.244495: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 2780160885630659466
2023-10-24 13:49:25.245656: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14411712073822479182


INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-24_13-42-39/assets


...

7851/7851 [==============================] - 389s 50ms/step - loss: 0.3337 - accuracy: 0.8545 - auc: 0.9316 - val_loss: 0.5239 - val_accuracy: 0.7820 - val_auc: 0.8740 - lr: 0.0010
Epoch 9/10
7850/7851 [============================>.] - ETA: 0s - loss: 0.3173 - accuracy: 0.8628 - auc: 0.9383INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-24_13-42-39/assets


INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-24_13-42-39/assets


7851/7851 [==============================] - 388s 49ms/step - loss: 0.3173 - accuracy: 0.8628 - auc: 0.9383 - val_loss: 0.5178 - val_accuracy: 0.7826 - val_auc: 0.8754 - lr: 0.0010
Epoch 10/10
7851/7851 [==============================] - ETA: 0s - loss: 0.3025 - accuracy: 0.8698 - auc: 0.9440INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-24_13-42-39/assets


INFO:tensorflow:Assets written to: data/models/resnet-gender-2023-10-24_13-42-39/assets


7851/7851 [==============================] - 390s 50ms/step - loss: 0.3025 - accuracy: 0.8698 - auc: 0.9440 - val_loss: 0.5050 - val_accuracy: 0.7920 - val_auc: 0.8769 - lr: 0.0010


In [9]:
model.save("data/models/resnet-gender.keras")